In [1]:
import torch
import MinkowskiEngine as ME

data = [
    [0, 0, 2.1, 0, 0],
    [0, 1, 1.4, 3, 0],
    [0, 0, 4.0, 0, 0]
]

def to_sparse_coo(data):
    # An intuitive way to extract coordinates and features
    coords, feats = [], []
    for i, row in enumerate(data):
        for j, val in enumerate(row):
            if val != 0:
                coords.append([i, j])
                feats.append([val])
    return torch.IntTensor(coords), torch.FloatTensor(feats)

to_sparse_coo(data)

(tensor([[0, 2],
         [1, 1],
         [1, 2],
         [1, 3],
         [2, 2]], dtype=torch.int32),
 tensor([[2.1000],
         [1.0000],
         [1.4000],
         [3.0000],
         [4.0000]]))

In [11]:
import torch
import MinkowskiEngine as ME
import numpy as np

pc_1 = np.random.rand(8192,3) * 20
voxel_size = 0.1

# Voxelization
_, sel1 = ME.utils.sparse_quantize(pc_1 / voxel_size, return_index=True)

# Slect the voxelized points
pc_1 = pc_1[sel1,:]

# Get sparse indices
coords1 = np.floor(pc_1 / voxel_size)

# Use absolute features as input
feats1 = coords1.copy()

coords_batch1, feats_batch1 = ME.utils.sparse_collate(coords=[coords1], feats=[feats1])
print(type(feats_batch1))
sinput1 = ME.SparseTensor(features=feats_batch1,
            coordinates=coords_batch1)
sinput1_cuda = ME.SparseTensor(features=feats_batch1.to('cuda'),
            coordinates=coords_batch1.to('cuda'))
print(sinput1_cuda.F)
# for b_idx in range(len(sinput1.decomposed_coordinates)):
#     feat_s = sinput1.F[sinput1.C[:,0] == b_idx]

#     print(sum(sinput1.C[:,0] == b_idx))
#     print(sinput1.F.shape)
#     print(feat_s.shape)

# for b_idx in range(len(sinput1_cuda.decomposed_coordinates)):
#     feat_s_cuda = sinput1_cuda.F[sinput1_cuda.C[:,0] == b_idx]

#     print(sum(sinput1_cuda.C[:,0] == b_idx))
#     print(sinput1_cuda.F.shape)
#     print(feat_s_cuda.shape)

<class 'torch.Tensor'>
tensor([[143.,  52.,  75.],
        [121.,  97., 114.],
        [ 36.,  32.,  86.],
        ...,
        [ 66.,  30., 134.],
        [ 82., 151., 182.],
        [121.,  99., 113.]], device='cuda:0', dtype=torch.float64)
